In [1]:
#!pip install geopandas pandas shapely

In [2]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point
import os
caminho = 'C:\\Users\\Edmilson\\Gama_Workspace2\\ABMS-WP'

# 1. Load the census tract shapefile
setores = gpd.read_file(os.path.join(caminho,'includes\\maps\\itapua\\Mapa_Itapua.shp'))

# 2. Select only the CD_SETOR column and the geometry
setores = setores[['CD_SETOR', 'geometry']]

# 3. Displaying the polygon and its respective sectors
setores.head()

,CD_SETOR,geometry
0,292740805090020,"POLYGON ((-38.36855 -12.94796, -38.36859 -12.9..."
1,292740805090021,"POLYGON ((-38.36738 -12.94851, -38.36739 -12.9..."
2,292740805090022,"POLYGON ((-38.36442 -12.94586, -38.36457 -12.9..."
3,292740805090023,"POLYGON ((-38.36433 -12.94755, -38.36433 -12.9..."
4,292740805090024,"POLYGON ((-38.36165 -12.94856, -38.36168 -12.9..."


In [3]:
# 4. Upload the CSV file with the supply points
pontos_df = pd.read_csv(os.path.join(caminho,'includes\\Tabela_consumidores_Itapua_convertida.csv'), sep=';')

# 5. Display supply points and their coordinates
pontos_df.head()

,SK_MATRICULA,NM_LOCALIDADE,NM_CATEGORIATARIFARIA,NM_SITUACAO_IMOVEL,NN_MORADORES,ST_PISCINA,LAT_GEO,LONG_GEO,X,Y
0,6E60FF997A2A7FD7665AC40BAD9BE286D7C47A1C3521CE...,SALVADOR UMB,RESIDENCIAL,HABITADO,1,0,-12.938243,-38.359220,569505.509777,8.569607e+06
1,E6195B23791D174F7C612B6324CACDED0CBDC2283FC728...,SALVADOR UMB,RESIDENCIAL,HABITADO,0,0,-12.934564,-38.355526,569907.238405,8.570013e+06
2,0C2ED22E52F7B889C5DCB715B44E40C0CF4E25A77F1A17...,SALVADOR UMB,RESIDENCIAL,HABITADO,0,0,-12.940582,-38.362400,569159.917133,8.569349e+06
3,6D38EE4CEC85F41B626000179F50D3599EEFA9B20AC328...,SALVADOR UMB,RESIDENCIAL,HABITADO,0,0,-12.944986,-38.370763,568251.558470,8.568864e+06
4,0420BAE41A9CA4FBA6CE1F1E120C73D05448330C834D72...,SALVADOR UMB,RESIDENCIAL,HABITADO,0,0,-12.940044,-38.359239,569502.949980,8.569408e+06


In [4]:

# 6. Convert the Points DataFrame to a GeoDataFrame
geometry = [Point(xy) for xy in zip(pontos_df['LONG_GEO'], pontos_df['LAT_GEO'])]
pontos_gdf = gpd.GeoDataFrame(pontos_df, geometry=geometry, crs="EPSG:4326")  # EPSG:4326 é o código para WGS84


#7. Ensure both GeoDataFrames are in the same coordinate system
pontos_gdf = pontos_gdf.to_crs(setores.crs)

# 8. Perform the spatial join
# This will add the census sector information (geometry and CD_SETOR) to each supply point
resultado = gpd.sjoin(pontos_gdf, setores, how="left", predicate="within")

#9. Delete unnecessary columns
resultado.drop(columns=['geometry','index_right'],inplace=True)

# 10. Delete lines that do not belong to the Itapuã census sectors
resultado.dropna(subset=['CD_SETOR'], inplace=True)

# 11. Save the result to a new CSV (optional)
resultado.to_csv(os.path.join(caminho,'includes\\Tabela_consumidores_Itapua_com_setor.csv'), index=False)


# 12. View the first few lines of the result
print(resultado.head())

                                        SK_MATRICULA NM_LOCALIDADE  \
0  6E60FF997A2A7FD7665AC40BAD9BE286D7C47A1C3521CE...  SALVADOR UMB   
1  E6195B23791D174F7C612B6324CACDED0CBDC2283FC728...  SALVADOR UMB   
2  0C2ED22E52F7B889C5DCB715B44E40C0CF4E25A77F1A17...  SALVADOR UMB   
3  6D38EE4CEC85F41B626000179F50D3599EEFA9B20AC328...  SALVADOR UMB   
4  0420BAE41A9CA4FBA6CE1F1E120C73D05448330C834D72...  SALVADOR UMB   

  NM_CATEGORIATARIFARIA NM_SITUACAO_IMOVEL  NN_MORADORES  ST_PISCINA  \
0           RESIDENCIAL           HABITADO             1           0   
1           RESIDENCIAL           HABITADO             0           0   
2           RESIDENCIAL           HABITADO             0           0   
3           RESIDENCIAL           HABITADO             0           0   
4           RESIDENCIAL           HABITADO             0           0   

     LAT_GEO   LONG_GEO              X             Y         CD_SETOR  
0 -12.938243 -38.359220  569505.509777  8.569607e+06  292740805090166  
1 

In [5]:
resultado.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Index: 15297 entries, 0 to 15331
Data columns (total 11 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   SK_MATRICULA           15297 non-null  object 
 1   NM_LOCALIDADE          15297 non-null  object 
 2   NM_CATEGORIATARIFARIA  15297 non-null  object 
 3   NM_SITUACAO_IMOVEL     15297 non-null  object 
 4   NN_MORADORES           15297 non-null  int64  
 5   ST_PISCINA             15297 non-null  int64  
 6   LAT_GEO                15297 non-null  float64
 7   LONG_GEO               15297 non-null  float64
 8   X                      15297 non-null  float64
 9   Y                      15297 non-null  float64
 10  CD_SETOR               15297 non-null  object 
dtypes: float64(4), int64(2), object(5)
memory usage: 1.4+ MB
